In [1]:
#default_exp matching_zsl

In [2]:
#export
import os
import ast
import tqdm
import json
import attr
from operator import itemgetter
from scipy.stats import hmean

import concurrent.futures

import itertools


import pandas as pd
import numpy as np
from sklearn import feature_extraction, metrics, model_selection

import matplotlib.pyplot as plt
import gensim


from functools import partial

from mlutil.feature_extraction import embeddings
import mlutil
from scarce_learn import zero_shot
from scarce_learn.zero_shot import devise_jax, devise_torch
from github_search import paperswithcode_tasks, github_readmes, python_call_graph

In [3]:
%env XLA_PYTHON_CLIENT_PREALLOCATE=false

env: XLA_PYTHON_CLIENT_PREALLOCATE=false


In [4]:
# upstream

import_corpus_path = 'output/module_corpus.csv'
word_vectors_filename = 'output/import2vec_module_vectors.bin'

In [5]:
%cd ..

/home/kuba/Projects/github_search


%%time
import_corpus_df = pd.read_csv(import_corpus_path)
per_repo_imports = import_corpus_df.groupby('repo')['imports'].agg(sum).apply(set)
import_corpus_df['imports'] = import_corpus_df['imports'].apply(ast.literal_eval)

In [6]:
%%time
python_files_df = pd.read_csv('data/crawled_python_files.csv', encoding='latin-1')
repo_names = python_files_df['repo_name']
import_corpus_df = pd.read_csv(import_corpus_path)
per_repo_imports = import_corpus_df.groupby('repo')['imports'].agg(sum).apply(set)

CPU times: user 3min 30s, sys: 3.61 s, total: 3min 33s
Wall time: 3min 35s


In [7]:
python_files_df.shape

(1797972, 3)

In [8]:
import_corpus_df.shape

(1749175, 3)

In [9]:
python_files_df['repo_name']

0                   trangvu/ape-npi
1                   trangvu/ape-npi
2                   trangvu/ape-npi
3                   trangvu/ape-npi
4                   trangvu/ape-npi
                     ...           
1797967    vuanhtu1993/Keras-SRGANs
1797968    vuanhtu1993/Keras-SRGANs
1797969    vuanhtu1993/Keras-SRGANs
1797970    vuanhtu1993/Keras-SRGANs
1797971    vuanhtu1993/Keras-SRGANs
Name: repo_name, Length: 1797972, dtype: object

In [10]:
python_files_df['repo_name'].unique().shape

(26999,)

python_files_df['repo'] = python_files_df['repo_name'].str.split("/").apply(itemgetter(1))  + '/' + python_files_df['repo_name']
repo_names_tmp = python_files_df['repo_name']
repo_names = repo_names_tmp.unique()
python_files_df['repo_name'] = python_files_df['repo']
python_files_df['repo'] = repo_names_tmp

In [11]:
%%time
import2vec = gensim.models.KeyedVectors.load(word_vectors_filename)
import2vec_embedder = mlutil.feature_extraction.embeddings.AverageWordEmbeddingsVectorizer(import2vec)

CPU times: user 13.2 ms, sys: 0 ns, total: 13.2 ms
Wall time: 85.6 ms


In [12]:
task_name = "3d reconstruction"

In [13]:
import re
from github_search.paperswithcode_tasks import clean_task_name

In [14]:
re.sub(r"\d+d", "", task_name)

' reconstruction'

In [15]:
paperswithcode_with_imports_df = pd.read_csv('output/papers_with_imports.csv')
paperswithcode_with_imports_df['tasks'] = paperswithcode_with_imports_df['tasks'].apply(clean_task_name).apply(ast.literal_eval)
paperswithcode_with_imports_df['imports'] = paperswithcode_with_imports_df['imports'].str.replace("set\(\)", "{}").apply(ast.literal_eval)#str.replace("2d ", "").str.replace("3d ", "").str.replace("4d ", "").str.replace("6d ", "").str.lower().apply(ast.literal_eval)

In [153]:
paperswithcode_with_imports_df.shape

(17388, 26)

In [156]:
graphsage_data_train.X

1                                       phnk/D7047E
2                        Aiyoj/Thumbnail-Generation
3                                   tzt101/MichiGAN
4                               fairlearn/fairlearn
5                                 hrlblab/CircleNet
                            ...                    
17380                       allenai/break-evaluator
17381                        YeongHyeon/f-AnoGAN-TF
17382                               LeeDoYup/AnoGAN
17383                            tkwoo/anogan-keras
17387    Kano-Wu/Domain-Adversarial-Neural-Networks
Name: repo, Length: 14427, dtype: object

In [17]:
paperswithcode_with_imports_df['n_imports'] = paperswithcode_with_imports_df['imports'].apply(len) 

In [18]:
paperswithcode_with_imports_df['n_imports_with_embeddings'] = paperswithcode_with_imports_df['imports'].apply(lambda imps: len([imp in import2vec.vocab.keys() for imp in imps]))

In [19]:
%%time
word_embeddings = mlutil.feature_extraction.embeddings.load_gensim_embedding_model('glove-wiki-gigaword-300')

CPU times: user 32.9 s, sys: 215 ms, total: 33.1 s
Wall time: 33.4 s


In [20]:
import fasttext
fasttext_model = fasttext.load_model("output/python_files_fasttext_dim200.bin")

In [21]:
from gensim.models.callbacks import CallbackAny2Vec

        
class LossCallback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [159]:
paperswithcode_with_features_df['most_common_task'][:None]

0            dictionary learning
1             sentiment analysis
2                region proposal
3               image generation
4                       fairness
                  ...           
17383          anomaly detection
17384             style transfer
17385             style transfer
17386             style transfer
17387    representation learning
Name: most_common_task, Length: 17388, dtype: object

In [22]:
python_word_embeddings = gensim.models.Word2Vec.load('output/abstract_readme_w2v200.bin')

INFO - 20:40:50: loading wv recursively from output/abstract_readme_w2v200.bin.wv.* with mmap=None
INFO - 20:40:50: loading vectors from output/abstract_readme_w2v200.bin.wv.vectors.npy with mmap=None
INFO - 20:40:50: setting ignored attribute vectors_norm to None
INFO - 20:40:50: loading vocabulary recursively from output/abstract_readme_w2v200.bin.vocabulary.* with mmap=None
INFO - 20:40:50: loading trainables recursively from output/abstract_readme_w2v200.bin.trainables.* with mmap=None
INFO - 20:40:50: loading syn1neg from output/abstract_readme_w2v200.bin.trainables.syn1neg.npy with mmap=None
INFO - 20:40:50: setting ignored attribute cum_table to None
INFO - 20:40:50: loaded output/abstract_readme_w2v200.bin


In [23]:
#export

@attr.s
class RepoTaskData:
    
    tasks = attr.ib()
    repos = attr.ib()
    X = attr.ib()
    all_tasks = attr.ib()
    y = attr.ib()
    
    def split_tasks(area_grouped_tasks, test_size=0.2):
        tasks_train, tasks_test = model_selection.train_test_split(area_grouped_tasks['task'], stratify=area_grouped_tasks['area'], test_size=test_size, random_state=0)
        return tasks_train, tasks_test
    
    def create_split(tasks_train, all_tasks, paperswithcode_with_features_df, X_repr):
        train_indicator = paperswithcode_with_features_df['most_common_task'].isin(tasks_train)
        repos_train = paperswithcode_with_features_df['repo'][train_indicator]
        repos_test = paperswithcode_with_features_df['repo'][~train_indicator]
        X_repr = X_repr.apply(lambda x: " ".join(x))
        X_train = X_repr[train_indicator]
        X_test = X_repr[~train_indicator]
        all_tasks_train = all_tasks[train_indicator]
        all_tasks_test = all_tasks[~train_indicator]
        y_train = paperswithcode_with_features_df[train_indicator]['most_common_task'].str.lower().apply(clean_task_name)
        y_test = paperswithcode_with_features_df[~train_indicator]['most_common_task'].str.lower().apply(clean_task_name)
        
        return (
            RepoTaskData(tasks_train, repos_train, X_train, all_tasks_train, y_train),
            RepoTaskData(tasks_test, repos_test, X_test, all_tasks_test, y_test)
        )

In [24]:
#export


def get_first_vocab_entry(vocab):
    return list(itertools.islice(vocab.items(), 1))[0][0] 


class PairedKeyedVectors:
    
    @attr.s
    class wv:
        vocab = attr.ib()
    
    def __init__(self, kv1, kv2):
        self.kv1 = kv1
        self.kv2 = kv2
        self.vocab = {**kv1.vocab, **kv2.vocab} 
        self.dim1 = len(kv1[get_first_vocab_entry(kv1.vocab)])
        self.dim2 = len(kv2[get_first_vocab_entry(kv2.vocab)])
        self.wv= PairedKeyedVectors.wv(self.vocab)
    
    def __getitem__(self, item):
        if not item in self.kv1.vocab.keys():
            return np.concatenate([np.zeros(self.dim1), self.kv2[item]])
        elif not item in self.kv2.vocab.keys():
            return np.concatenate([self.kv1[item], np.zeros(self.dim2)])
        else:
            return np.concatenate([self.kv1[item], self.kv2[item]])
    


@attr.s
class RetrieverLearner:
    
    zs_learner: zero_shot.ZeroShotClassifier = attr.ib()
    input_embedder: embeddings.EmbeddingVectorizer = attr.ib() 
    y_embedder: embeddings.EmbeddingVectorizer = attr.ib()
    input_embedder_kwargs = attr.ib(default=dict())
        
    @staticmethod
    def create(
        zs_learner: zero_shot.ZeroShotClassifier,
        input_embeddings: gensim.models.KeyedVectors,
        target_embeddings: gensim.models.KeyedVectors,
        input_embedding_method: embeddings.EmbeddingVectorizer,
        y_embedding_method: embeddings.EmbeddingVectorizer,
        input_embedder_kwargs=dict()
    ):
        input_embedder = input_embedding_method(input_embeddings, **input_embedder_kwargs) 
        y_embedder = y_embedding_method(target_embeddings)
        return RetrieverLearner(zs_learner, input_embedder, y_embedder)
    
    def get_target_embeddings(self, y):
        unique_y = pd.Series(y.unique())
        y_embeddings = self.y_embedder.transform(unique_y)
        return unique_y, y_embeddings
    
    def fit_learner(self, data, **kwargs):
        self.input_embedder.fit(data.X)
        X_embeddings = self.input_embedder.transform(data.X)
        self.y_embedder.fit(data.y)
        unique_y, y_embeddings = self.get_target_embeddings(data.y)
        input_y_idxs = data.y.apply(lambda t: unique_y[unique_y == t].index[0])
        self.zs_learner.fit(np.array(X_embeddings), np.array(input_y_idxs), np.array(y_embeddings), **kwargs)
        
    def predict_idxs(self, X, y_embeddings):
        X_embeddings = self.input_embedder.transform(X)
        return self.zs_learner.predict(X_embeddings, y_embeddings)
    
    def predict_topk(self, X, y_embeddings, target_names, k=5, similarity=metrics.pairwise.cosine_similarity):
        X_embeddings = self.input_embedder.transform(X)
        predictions = self.zs_learner.predict_raw(X_embeddings)
        target_similarities = similarity(predictions, y_embeddings)
        targets = [target_names[row[:k]] for row in (-target_similarities).argsort(axis=1)]
        return targets
        
    def evaluate(self, data, metric):
        unique_y, y_embeddings = self.get_target_embeddings(data.y)
        input_y_idxs = data.y.apply(lambda t: unique_y[unique_y == t].index[0])
        predicted_idxs = self.predict_idxs(data.X, y_embeddings)
        return metric(input_y_idxs, predicted_idxs)

In [25]:
#export

def get_accuracy(learner, X, y, y_names, k=10, similarity=metrics.pairwise.cosine_similarity):
    input_embeddings = learner.input_embedder.transform(X)
    y_embeddings = learner.y_embedder.transform(y_names)
    predictions = learner.zs_learner.predict_raw(input_embeddings)
    target_similarities = similarity(predictions, y_embeddings)
    target_idxs = (-target_similarities).argsort(axis=1)
    targets = [y_names.iloc[row[:k]] for row in target_idxs]

    accuracies = np.zeros(len(X))
    for i in range(len(X)):
        true_tasks = set(all_tasks_test.iloc[i])
        accuracies[i] = len(true_tasks.intersection(set(targets[i].values))) / min(len(true_tasks), k)
    return accuracies.mean()

In [26]:
import pickle

graph = pickle.load(open('output/call_igraph.pkl', 'rb'))

In [27]:
len(graph.get_vertex_dataframe().iloc[graph.neighborhood(vertices=["<ROOT>"])[0]])

27000

get repos that are in graph 

In [28]:
graph_nodes = graph.get_vertex_dataframe()['name'].unique()

In [29]:
paperswithcode_with_imports_df.shape

(17388, 25)

In [30]:
%%time
paperswithcode_with_features_df = paperswithcode_with_imports_df[
    paperswithcode_with_imports_df['repo'].isin(graph.get_vertex_dataframe()['name']) |
    paperswithcode_with_imports_df['repo'].apply(lambda s: s.split("/")[1]).isin(graph.get_vertex_dataframe()['name'])
]

CPU times: user 897 ms, sys: 63.9 ms, total: 961 ms
Wall time: 966 ms


In [31]:
paperswithcode_with_imports_df.shape

(17388, 25)

In [32]:
paperswithcode_with_imports_df = paperswithcode_with_imports_df[paperswithcode_with_imports_df['repo'].isin(paperswithcode_with_features_df['repo'])]

In [33]:
paperswithcode_with_imports_df.shape

(17388, 25)

In [34]:
paperswithcode_with_features_df['most_common_task'] = paperswithcode_with_features_df['most_common_task'].str.lower()
tasks = paperswithcode_with_features_df['most_common_task'].str.lower()
tasks = tasks.apply(clean_task_name)
all_tasks = paperswithcode_with_features_df['tasks'].apply(lambda s: [clean_task_name(t) for t in s])
paperswithcode_with_features_df.shape

(17388, 25)

In [35]:
tasks = all_tasks.explode().drop_duplicates()

In [36]:
tasks

0                          dictionary learning
1                     named entity recognition
1                           sentiment analysis
2                              region proposal
2        user constrained thumbnail generation
                         ...                  
17209     image compression artifact reduction
17233      multi modal document classification
17280                      geometry perception
17311                       split and rephrase
17322      one shot image to image translation
Name: tasks, Length: 1311, dtype: object

In [37]:
#export

def get_area_grouped_tasks(paperswithcode_tasks_path='data/paperswithcode_tasks.csv'):
    area_grouped_tasks = pd.read_csv('data/paperswithcode_tasks.csv').dropna()
    area_grouped_tasks['task'] = area_grouped_tasks['task'].apply(clean_task_name)
    area_grouped_tasks = area_grouped_tasks[area_grouped_tasks['task'].isin(tasks)]
    area_counts = area_grouped_tasks['area'].value_counts()
    area_grouped_tasks = area_grouped_tasks[area_grouped_tasks['area'].isin(area_counts.index[area_counts > 1])]
    return area_grouped_tasks

In [38]:
get_area_grouped_tasks()['area'].value_counts()

INFO - 20:40:57: Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 20:40:57: NumExpr defaulting to 8 threads.


computer-vision                316
miscellaneous                  296
natural-language-processing    223
methodology                    108
medical                         72
speech                          36
time-series                     34
graphs                          33
playing-games                   23
robots                          20
computer-code                   15
knowledge-base                  15
music                           12
audio                           12
reasoning                       10
adversarial                      6
Name: area, dtype: int64

In [39]:
area_grouped_tasks = get_area_grouped_tasks()
area_grouped_tasks['task'] = area_grouped_tasks['task'].apply(clean_task_name)

In [40]:
tasks_train, tasks_test = RepoTaskData.split_tasks(area_grouped_tasks)

In [41]:
len(tasks_train)

984

In [42]:
tasks_test.str.contains("-").mean()

0.0

In [43]:
len(tasks_test)

247

In [44]:
paperswithcode_with_features_df['most_common_task']

0            dictionary learning
1             sentiment analysis
2                region proposal
3               image generation
4                       fairness
                  ...           
17383          anomaly detection
17384             style transfer
17385             style transfer
17386             style transfer
17387    representation learning
Name: most_common_task, Length: 17388, dtype: object

In [45]:
paperswithcode_with_features_df['most_common_task'].isin(tasks_test).sum()

4139

In [46]:
paperswithcode_with_features_df.shape

(17388, 25)

In [47]:
paperswithcode_with_features_df.shape

(17388, 25)

In [48]:
def get_readme_summaries(upstream, product, keywords=True):
    pool = concurrent.futures.ProcessPoolExecutor(max_workers=10)
    raw_readmes = list(pool.map(github_readmes.get_readme, paperswithcode_with_features_df['repo']))
    readmes = pd.Series(raw_readmes).apply(github_readmes.try_decode)
    return readmes

In [49]:
def get_readmes(df, keywords=True):
    pool = concurrent.futures.ProcessPoolExecutor(max_workers=10)
    raw_readmes = list(pool.map(github_readmes.get_readme, df['repo']))
    readmes = list(map(github_readmes.try_decode, raw_readmes))
    return readmes

In [50]:
paperswithcode_with_features_df.columns

Index(['Unnamed: 0', 'paper_url', 'arxiv_id', 'title', 'abstract', 'url_abs',
       'url_pdf', 'proceeding', 'authors', 'tasks', 'date', 'methods',
       'framework', 'mentioned_in_github', 'mentioned_in_paper',
       'paper_arxiv_id', 'paper_title', 'paper_url_abs', 'paper_url_pdf',
       'repo', 'repo_url', 'most_common_task', 'imports', 'n_imports',
       'n_imports_with_embeddings'],
      dtype='object')

In [51]:
paperswithcode_with_features_df.shape

(17388, 25)

%%time
readmes = get_readmes(paperswithcode_with_features_df)

In [52]:
def try_keywords(text):
    return python_call_graph.try_run(gensim.summarization.keywords)(text)

In [53]:
pool = concurrent.futures.ProcessPoolExecutor(max_workers=20)

%%time
readme_keywords = pd.Series(pool.map(try_keywords, readmes)).str.replace("\n", " ")

In [54]:
dependency_records_df = pd.read_csv('output/processed_dependency_records.csv').dropna()#.iloc[:1000000]
non_root_dependency_records_df = dependency_records_df[
    (dependency_records_df['source'] != "<ROOT>") &
    (dependency_records_df['edge_type'] != 'repo-repo')
]

In [55]:
task_embedder = mlutil.feature_extraction.embeddings.AverageWordEmbeddingsVectorizer(word_embeddings)

In [56]:
#export


def get_outgoing_edges(graph, node):
    #idx = pd.Index(graph.names).get_loc(node)
    #outgoing_edges_idx = np.where(graph.mat[idx].todense())[1]
    return graph.get_vertex_dataframe().iloc[graph.successors(node)]['name']
    #return graph.names[outgoing_edges_idx]


def get_repo_functions(graph, repo):
    return ' '.join(get_outgoing_edges(graph, repo).values)

In [57]:
graph_records = pd.read_csv('output/dependency_records.csv')

In [58]:
#export


def prepare_task_train_test_split(upstream, area_grouped_tasks_path, product):
    area_grouped_tasks = get_area_grouped_tasks(area_grouped_tasks_path)
    tasks_train, tasks_test = RepoTaskData.split_tasks(area_grouped_tasks)
    tasks_train.to_csv(product['train'], index=None)
    tasks_test.to_csv(product['test'], index=None)


def prepare_graph_repo_task_data(upstream, product):
    graph_data_train, graph_data_test = RepoTaskData.create_split(tasks_train, all_tasks, paperswithcode_with_features_df, paperswithcode_with_imports_df['imports'])
    graph_data_train.X = graph_data_train.repos.apply(lambda x: get_repo_functions(graph, x))
    graph_data_test.X = graph_data_test.repos.apply(lambda x: get_repo_functions(graph, x))
    pickle.dump((graph_data_train, graph_data_test), open(str(product), "wb"))

In [59]:
%%time
if os.path.exists("output/tmp_graph_data.pkl"):
    (graph_data_train, graph_data_test) = pickle.load(open("output/tmp_graph_data.pkl", "rb"))
else:
    graph_data_train, graph_data_test = RepoTaskData.create_split(tasks_train, all_tasks, paperswithcode_with_features_df, paperswithcode_with_imports_df['imports'])
    graph_data_train.X = graph_data_train.repos.apply(lambda x: get_repo_functions(graph, x))
    graph_data_test.X = graph_data_test.repos.apply(lambda x: get_repo_functions(graph, x))
    pickle.dump((graph_data_train, graph_data_test), open("output/tmp_graph_data.pkl", "wb"))

CPU times: user 4.82 ms, sys: 16.3 ms, total: 21.1 ms
Wall time: 95.6 ms


In [60]:
get_outgoing_edges(graph, get_outgoing_edges(graph, graph_data_train.repos.iloc[0]).iloc[0])

vertex ID
627616                                  plot
635979    canbakiskan/neuro-inspired-defense
Name: name, dtype: object

In [61]:
len(graph_data_train.X)

14350

In [62]:
for i in range(len(graph_data_train.X)):
    graph_data_train.X.iloc[i] = graph_data_train.X.iloc[i].replace(graph_data_train.repos.iloc[i], "")
for i in range(len(graph_data_test.X)):
    graph_data_test.X.iloc[i] = graph_data_test.X.iloc[i].replace(graph_data_test.repos.iloc[i], "")

In [63]:
graph_data_train.X = graph_data_train.X.str.replace(":", " ")
graph_data_train.X = graph_data_train.X.str.replace("<ROOT>", " ")
graph_data_test.X = graph_data_test.X.str.replace(":", " ")
graph_data_test.X = graph_data_test.X.str.replace("<ROOT>", " ")

In [64]:
area_grouped_tasks['area'].value_counts()

computer-vision                316
miscellaneous                  296
natural-language-processing    223
methodology                    108
medical                         72
speech                          36
time-series                     34
graphs                          33
playing-games                   23
robots                          20
computer-code                   15
knowledge-base                  15
music                           12
audio                           12
reasoning                       10
adversarial                      6
Name: area, dtype: int64

In [65]:
#export
def maybe_get_ndarray_elem(arr, idx, default=-1):
    if len(arr) <= idx:
        return default
    else:
        return arr[idx]
    
    
def get_retrieval_results(learner, data, k=10, similarity=metrics.pairwise.cosine_similarity):
    y_names, __ = learner.get_target_embeddings(data.y)
    input_embeddings = learner.input_embedder.transform(data.X)
    y_embeddings = learner.y_embedder.transform(y_names)
    predictions = learner.zs_learner.predict_raw(input_embeddings)
    input_target_similarities = similarity(predictions, y_embeddings)

    X_recalled = [
        np.argsort(-input_target_similarities[:,y_idx])[:k]
        for (y_idx, __) in enumerate(y_names)
    ]
    return X_recalled


def get_retrieval_accuracies(learner, data, k=10, similarity=metrics.pairwise.cosine_similarity):
    y_names, __ = learner.get_target_embeddings(data.y)
    retrieved_X = get_retrieval_results(learner, data, k=k, similarity=similarity)
    retrieved_X_actual_labels = [data.all_tasks.iloc[idxs_recalled].explode().values for idxs_recalled in retrieved_X]
    retrieved_idxs = [
        np.where(retrieved_X_actual_labels[y_idx] == y_name)[0]
        for (y_idx, y_name) in enumerate(y_names)
    ]
    num_recalled = [len(r) for r in retrieved_idxs]
    pos_recalled = [maybe_get_ndarray_elem(r, 0) for r in retrieved_idxs] 
    accurately_recalled = [r > -1 for r in pos_recalled]
    return pd.DataFrame({"retrieved_labels": retrieved_X_actual_labels, "num_recalled": num_recalled, "recalled": accurately_recalled, "position": pos_recalled}, index=y_names)


def get_retrieval_accuracy(learner, data, k=10, similarity=metrics.pairwise.cosine_similarity):
    y_names, __ = learner.get_target_embeddings(data.y)
    return np.mean(get_retrieval_accuracies(learner, data, k, similarity)['recalled'])

In [66]:
np.where(pd.Series([1,2,3]).values == 2)[0]

array([1])

In [67]:
#export


def run_learner_experiment(
    retriever_learner,
    data_train, data_test
):
    retriever_learner.fit_learner(data_train)
    
    accuracy_train = retriever_learner.evaluate(data_train, metrics.accuracy_score)
    accuracy_test = retriever_learner.evaluate(data_test, metrics.accuracy_score)
    top10_accuracy_train = get_retrieval_accuracy(retriever_learner, data_train, k=10)
    top10_accuracy_test = get_retrieval_accuracy(retriever_learner, data_test, k=10)
    
    return dict(
        accuracy_train=accuracy_train,
        accuracy_test=accuracy_test,
        top10_accuracy_train=top10_accuracy_train,
        top10_accuracy_test=top10_accuracy_test
    )

## Abstracts

In [68]:
paperswithcode_with_imports_df['abstract']

0        Deep Neural Networks (DNNs) are vulnerable to ...
1        Word2Vec is a prominent model for natural lang...
2        Thumbnails are widely used all over the world ...
3        Despite the recent success of face image gener...
4        We present a systematic approach for achieving...
                               ...                        
17383    Obtaining models that capture imaging markers ...
17384    Gatys et al. recently introduced a neural algo...
17385    Gatys et al. recently introduced a neural algo...
17386    Gatys et al. recently introduced a neural algo...
17387    We introduce a new representation learning alg...
Name: abstract, Length: 17388, dtype: object

In [69]:
has_abstract = ~paperswithcode_with_imports_df['abstract'].isna()

In [70]:
tasks_train[has_abstract]
paperswithcode_with_features_df[has_abstract]

,Unnamed: 0,paper_url,arxiv_id,title,abstract,url_abs,url_pdf,proceeding,authors,tasks,...,paper_arxiv_id,paper_title,paper_url_abs,paper_url_pdf,repo,repo_url,most_common_task,imports,n_imports,n_imports_with_embeddings
0,0,https://paperswithcode.com/paper/a-neuro-inspi...,2011.10867,A Neuro-Inspired Autoencoding Defense Against ...,Deep Neural Networks (DNNs) are vulnerable to ...,https://arxiv.org/abs/2011.10867v2,https://arxiv.org/pdf/2011.10867v2.pdf,NaN,"['Can Bakiskan', 'Metehan Cekic', 'Ahmet Dunda...",[dictionary learning],...,2011.10867,A Neuro-Inspired Autoencoding Defense Against ...,https://arxiv.org/abs/2011.10867v2,https://arxiv.org/pdf/2011.10867v2.pdf,canbakiskan/neuro-inspired-defense,https://github.com/canbakiskan/neuro-inspired-...,dictionary learning,"{sklearn, namers, sys, torchvision, train_test...",30,30
1,3,https://paperswithcode.com/paper/word2vec-opti...,2003.11645,Word2Vec: Optimal Hyper-Parameters and Their I...,Word2Vec is a prominent model for natural lang...,https://arxiv.org/abs/2003.11645v2,https://arxiv.org/pdf/2003.11645v2.pdf,NaN,"['Tosin P. Adewumi', 'Foteini Liwicki', 'Marcu...","[named entity recognition, sentiment analysis]",...,2003.11645,Word2Vec: Optimal Hyper-Parameters and Their I...,https://arxiv.org/abs/2003.11645v2,https://arxiv.org/pdf/2003.11645v2.pdf,phnk/D7047E,https://github.com/phnk/D7047E,sentiment analysis,"{sklearn, copy, sigopt, tempfile, urllib, sys,...",76,76
2,4,https://paperswithcode.com/paper/user-constrai...,1810.13054,User Constrained Thumbnail Generation using Ad...,Thumbnails are widely used all over the world ...,http://arxiv.org/abs/1810.13054v3,http://arxiv.org/pdf/1810.13054v3.pdf,NaN,"['Perla Sai Raj Kishore', 'Ayan Kumar Bhunia',...","[region proposal, user constrained thumbnail g...",...,1810.13054,User Constrained Thumbnail Generation using Ad...,http://arxiv.org/abs/1810.13054v3,http://arxiv.org/pdf/1810.13054v3.pdf,Aiyoj/Thumbnail-Generation,https://github.com/Aiyoj/Thumbnail-Generation,region proposal,"{tensorflow, custom_vgg19, pandas, numpy, re, ...",15,15
3,5,https://paperswithcode.com/paper/michigan-mult...,2010.16417,MichiGAN: Multi-Input-Conditioned Hair Image G...,Despite the recent success of face image gener...,https://arxiv.org/abs/2010.16417v1,https://arxiv.org/pdf/2010.16417v1.pdf,NaN,"['Zhentao Tan', 'Menglei Chai', 'Dongdong Chen...","[conditional image generation, image generation]",...,2010.16417,MichiGAN: Multi-Input-Conditioned Hair Image G...,https://arxiv.org/abs/2010.16417v1,https://arxiv.org/pdf/2010.16417v1.pdf,tzt101/MichiGAN,https://github.com/tzt101/MichiGAN,image generation,"{sys, PyQt5, threading, trainers, importlib, t...",40,40
4,6,https://paperswithcode.com/paper/a-reductions-...,1803.02453,A Reductions Approach to Fair Classification,We present a systematic approach for achieving...,http://arxiv.org/abs/1803.02453v3,http://arxiv.org/pdf/1803.02453v3.pdf,ICML 2018 7,"['Alekh Agarwal', 'Alina Beygelzimer', 'Mirosl...",[fairness],...,1803.02453,A Reductions Approach to Fair Classification,http://arxiv.org/abs/1803.02453v3,http://arxiv.org/pdf/1803.02453v3.pdf,fairlearn/fairlearn,https://github.com/fairlearn/fairlearn,fairness,"{copy, _mean_predictions, subprocess, time, co...",82,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17383,30798,https://paperswithcode.com/paper/unsupervised-...,1703.05921,Unsupervised Anomaly Detection with Generative...,Obtaining models that capture imaging markers ...,http://arxiv.org/abs/1703.05921v1,http://arxiv.org/pdf/1703.05921v1.pdf,NaN,"['Thomas Schlegl', 'Philipp Seeböck', 'Sebasti...","[anomaly detection, unsupervised anomaly detec...",...,1703.05921,Unsupervised Anomaly Detection with Generative...,http://arxiv.org/abs/1703.05921v1,http://arxiv.org/pdf/1703.05921v1.pdf,tkwoo/anogan-keras,https://github.com/tkwoo/anogan-keras,anomaly detection,"{tensorflow, matplotlib, sklearn, numpy, cv2, ...",12,12
17384,30799,https://paperswithcode.com/pa

In [71]:
abstract_data_train, abstract_data_test = RepoTaskData.create_split(tasks_train[has_abstract], all_tasks[has_abstract], paperswithcode_with_features_df[has_abstract], paperswithcode_with_features_df[has_abstract]['abstract'].str.split())

In [72]:
from scarce_learn.zero_shot import devise_jax

In [73]:
abstract_learner = RetrieverLearner.create(
    zero_shot.ESZSLearner(100, 10),
    python_word_embeddings,
    python_word_embeddings,
    embeddings.AverageWordEmbeddingsVectorizer,
    embeddings.AverageWordEmbeddingsVectorizer
)

abstract_learner.fit_learner(abstract_data_train)

In [74]:
run_learner_experiment(abstract_learner, abstract_data_train, abstract_data_test)

{'accuracy_train': 0.5326790561703905,
 'accuracy_test': 0.13952702702702702,
 'top10_accuracy_train': 0.8793103448275862,
 'top10_accuracy_test': 0.7355371900826446}

# Abstract model using fasttext trained on Python code

In [75]:
ezslearner = zero_shot.ESZSLearner()
abstract_fasttext_learner = RetrieverLearner.create(
    zero_shot.ESZSLearner(100, 10),
    word_embeddings,
    fasttext_model,
    embeddings.AverageWordEmbeddingsVectorizer,
    embeddings.FastTextVectorizer
)

In [76]:
run_learner_experiment(abstract_fasttext_learner, abstract_data_train, abstract_data_test)

{'accuracy_train': 0.4578548061529895,
 'accuracy_test': 0.07128378378378378,
 'top10_accuracy_train': 0.6344827586206897,
 'top10_accuracy_test': 0.4297520661157025}

# word2vec model on READMEs

In [77]:
paperswithcode_with_readmes_df = pd.read_csv("output/papers_with_readmes.csv")

In [78]:
paperswithcode_with_imports_df['readme'] = paperswithcode_with_readmes_df['readme'] 
paperswithcode_with_features_df['readme'] = paperswithcode_with_readmes_df['readme'] 

In [79]:
has_readme = ~paperswithcode_with_imports_df['readme'].isna()

readme_data_train, readme_data_test = RepoTaskData.create_split(tasks_train[has_readme], all_tasks[has_readme], paperswithcode_with_features_df[has_readme], paperswithcode_with_features_df[has_readme]['readme'].str.split())

In [80]:
paperswithcode_with_features_df['tasks'].apply(" ".join).str.contains("-").mean()

0.0

In [81]:
tasks_train[has_readme].str.contains("-").mean()

0.0

In [82]:
all_tasks[has_readme].apply(" ".join).str.contains("-").mean()

0.0

In [83]:
readme_data_test.y.str.contains("-").mean()

0.0

In [84]:
readme_data_test.all_tasks.apply(" ".join).str.contains("-").mean()

0.0

In [85]:
paperswithcode_with_imports_df.columns

Index(['Unnamed: 0', 'paper_url', 'arxiv_id', 'title', 'abstract', 'url_abs',
       'url_pdf', 'proceeding', 'authors', 'tasks', 'date', 'methods',
       'framework', 'mentioned_in_github', 'mentioned_in_paper',
       'paper_arxiv_id', 'paper_title', 'paper_url_abs', 'paper_url_pdf',
       'repo', 'repo_url', 'most_common_task', 'imports', 'n_imports',
       'n_imports_with_embeddings', 'readme'],
      dtype='object')

In [86]:
readme_learner = RetrieverLearner.create(
    zero_shot.ESZSLearner(100, 100),
    python_word_embeddings,
    python_word_embeddings,
    embeddings.AverageWordEmbeddingsVectorizer,
    embeddings.AverageWordEmbeddingsVectorizer
)

readme_learner.fit_learner(readme_data_train)

In [87]:
run_learner_experiment(readme_learner, readme_data_train, readme_data_test)

{'accuracy_train': 0.3888042650418888,
 'accuracy_test': 0.059114735002912054,
 'top10_accuracy_train': 0.8235294117647058,
 'top10_accuracy_test': 0.6204379562043796}

## Fasttext on READMEs - worse than word2vec

In [88]:
ezslearner = zero_shot.ESZSLearner()
readme_fasttext_learner = RetrieverLearner.create(
    zero_shot.ESZSLearner(100, 10),
    word_embeddings,
    fasttext_model,
    embeddings.AverageWordEmbeddingsVectorizer,
    embeddings.FastTextVectorizer
)

In [89]:
run_learner_experiment(readme_fasttext_learner, readme_data_train, readme_data_test)

{'accuracy_train': 0.27722772277227725,
 'accuracy_test': 0.028246942341292953,
 'top10_accuracy_train': 0.5625,
 'top10_accuracy_test': 0.39416058394160586}

# README keywords

readme_keywords_data_train, readme_keywords_data_test = RepoTaskData.create_split(tasks_train[has_readme], all_tasks[has_readme], paperswithcode_with_features_df[has_readme], readme_keywords[has_readme].str.split())

ezslearner = zero_shot.ESZSLearner()
readme_keywords_learner = RetrieverLearner.create(
    zero_shot.ESZSLearner(10, 10),
    word_embeddings,
    word_embeddings,
    embeddings.AverageWordEmbeddingsVectorizer,
    embeddings.AverageWordEmbeddingsVectorizer
)

run_learner_experiment(readme_keywords_learner, readme_keywords_data_train, readme_keywords_data_test)

## Import2Vec

In [90]:
import_data_train, import_data_test = RepoTaskData.create_split(tasks_train, all_tasks, paperswithcode_with_imports_df, paperswithcode_with_imports_df['imports'])

In [91]:
ezslearner = zero_shot.ESZSLearner()
import2vec_learner = RetrieverLearner.create(
    zero_shot.ESZSLearner(lmbda=100.0, gamma=10.0),
    import2vec,
    word_embeddings,
    embeddings.AverageWordEmbeddingsVectorizer,
    embeddings.AverageWordEmbeddingsVectorizer
)

run_learner_experiment(import2vec_learner, import_data_train, import_data_test)

{'accuracy_train': 0.5151515151515151,
 'accuracy_test': 0.0019014693171996544,
 'top10_accuracy_train': 1.0,
 'top10_accuracy_test': 0.0297029702970297}

## PRoNe

In [92]:
prone_embeddings = gensim.models.KeyedVectors.load("data/prone_embeddings.bin")

INFO - 20:45:55: loading Word2VecKeyedVectors object from data/prone_embeddings.bin
INFO - 20:45:58: loading vectors from data/prone_embeddings.bin.vectors.npy with mmap=None
INFO - 20:45:59: setting ignored attribute vectors_norm to None
INFO - 20:45:59: loaded data/prone_embeddings.bin


Using repo embedding from node embeddings

In [93]:
prone_learner = RetrieverLearner.create(
    zero_shot.ESZSLearner(100,10),
    prone_embeddings,
    python_word_embeddings,
    embeddings.AverageWordEmbeddingsVectorizer,
    embeddings.AverageWordEmbeddingsVectorizer
)

run_learner_experiment(prone_learner, graph_data_train, graph_data_test)

{'accuracy_train': 0.1716376306620209,
 'accuracy_test': 0.012179065174456879,
 'top10_accuracy_train': 0.4793103448275862,
 'top10_accuracy_test': 0.1557377049180328}

## GraphSage

## aggregating vertex embeddings 

In [94]:
graphsage_kv_file ="output/graphsage_embeddings_fasttext_dim200_epochs20_dim200_layers2.bin"

In [95]:
graphsage_embeddings = gensim.models.KeyedVectors.load(graphsage_kv_file)

INFO - 20:46:05: loading Word2VecKeyedVectors object from output/graphsage_embeddings_fasttext_dim200_epochs20_dim200_layers2.bin
INFO - 20:46:08: loading vectors from output/graphsage_embeddings_fasttext_dim200_epochs20_dim200_layers2.bin.vectors.npy with mmap=None
INFO - 20:46:09: setting ignored attribute vectors_norm to None
INFO - 20:46:09: loaded output/graphsage_embeddings_fasttext_dim200_epochs20_dim200_layers2.bin


In [96]:
list(graphsage_embeddings.vocab)[-1]

'coincidence'

In [97]:
vocab_list = list(graphsage_embeddings.vocab.keys())

In [98]:
tokens = graph_data_train.X[150].strip().split()

In [99]:
[token in vocab_list for token in tokens]

[True, True, True, True, True, True, True, True, True, True, True, True, True]

In [100]:
ezslearner = zero_shot.ESZSLearner()
graphsage_learner = RetrieverLearner.create(
    zero_shot.ESZSLearner(100, 10),
    graphsage_embeddings,
    python_word_embeddings,
    embeddings.AverageWordEmbeddingsVectorizer,
    embeddings.AverageWordEmbeddingsVectorizer
)

run_learner_experiment(graphsage_learner, graph_data_train, graph_data_test)

{'accuracy_train': 0.1616724738675958,
 'accuracy_test': 0.01053324555628703,
 'top10_accuracy_train': 0.5241379310344828,
 'top10_accuracy_test': 0.12295081967213115}

In [144]:
prone_results = run_learner_experiment(prone_learner, graph_data_train, graph_data_test)

In [101]:
abstract_results = run_learner_experiment(abstract_learner, abstract_data_train, abstract_data_test)
readme_results = run_learner_experiment(readme_learner, readme_data_train, readme_data_test)
graphsage_results = run_learner_experiment(graphsage_learner, graph_data_train, graph_data_test)

In [146]:
results_df = pd.DataFrame.from_records([abstract_results, readme_results, prone_results, graphsage_results])
results_df['method'] = ['abstract', 'readme', 'prone', 'graphsage']

In [147]:
results_df

,accuracy_train,accuracy_test,top10_accuracy_train,top10_accuracy_test,method
0,0.532679,0.139527,0.879310,0.735537,abstract
1,0.388804,0.059115,0.823529,0.620438,readme
2,0.171638,0.009875,0.479310,0.100000,prone
3,0.165523,0.013509,0.441379,0.297521,graphsage


## using GraphSAGE model for embedding

In [104]:
#export


class LambdaTransformer:
    
    def __init__(self, transform_fn):
        self.transform = transform_fn
        
    def fit(self, X, **kwargs):
        return self
    
    
class PyGGraphModelTransformer:
    
    def __init__(self, model, dependency_graph_wrapper):
        self.model = model
        self.dependency_graph_wrapper = dependency_graph_wrapper
        
    def transform(self, x):
        return self.dependency_graph_wrapper.get_vertex_embeddings(x, self.model)
    
    def fit(self, X, **kwargs):
        return self

In [105]:
from github_search.pytorch_geometric_data import PygGraphWrapper
import torch

In [106]:
fasttext_embedder = embeddings.FastTextVectorizer(fasttext_model)

In [107]:
%%time
dependency_graph_wrapper = PygGraphWrapper(fasttext_embedder, non_root_dependency_records_df, "source", "destination")

CPU times: user 43.9 s, sys: 3.86 s, total: 47.7 s
Wall time: 43.1 s


/home/kuba/Projects/github_search/github_search/pytorch_geometric_data.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.dataset = Data(torch.tensor(features), torch.tensor(edge_index))


In [108]:
!ls -ltr output/*graphsage*pth

-rw-rw-r-- 1 kuba kuba 166280 paź  7 19:44 output/graphsage_model.pth
-rw-rw-r-- 1 kuba kuba 166283 paź  7 20:19 output/residual_graphsage_model.pth
-rw-rw-r-- 1 kuba kuba 166282 paź 10 12:41 output/graphsage_model_no_root.pth
-rw-rw-r-- 1 kuba kuba 809626 paź 10 21:24 output/graphsage_model_50_dim200_layers3.pth
-rw-rw-r-- 1 kuba kuba 166283 paź 11 13:04 output/graphsage_model_100_dim100_layers2.pth
-rw-rw-r-- 1 kuba kuba 246219 paź 11 14:24 output/graphsage_model_50_dim100_layers2.pth
-rw-rw-r-- 1 kuba kuba 647053 paź 29 22:16 output/graphsage_model_60_dim200_layers2.pth
-rw-rw-r-- 1 kuba kuba 646987 paź 30 20:12 output/graphsage_model_2_dim200_layers2.pth
-rw-rw-r-- 1 kuba kuba 646988 paź 30 20:40 output/graphsage_model_40_dim200_layers2.pth
-rw-rw-r-- 1 kuba kuba 646987 paź 31 10:22 output/graphsage_model_100_dim200_layers2.pth
-rw-rw-r-- 1 kuba kuba 646986 paź 31 12:58 output/graphsage_model_200_dim200_layers2.pth
-rw-rw-r-- 1 kuba kuba 646987 lis  9 19:38 output/graphsage_model_1

In [109]:
graphsage_model = torch.load("output/graphsage_model_20_dim200_layers2.pth").cpu()
graphsage_model.training = False

In [110]:
graphsage_data_train, graphsage_data_test = RepoTaskData.create_split(tasks_train, all_tasks, paperswithcode_with_features_df, paperswithcode_with_imports_df.repo.apply(lambda s: [s]))

In [111]:
graph_data_test.y = graph_data_test.y.apply(clean_task_name)

In [112]:
graphsage_data_test.y.str.contains("-").mean()#= graph_data_test.y.apply(clean_task_name)

0.0

In [113]:
graphsage_data_test.all_tasks.apply(" ".join).str.contains("-").mean()#= graph_data_test.y.apply(clean_task_name)

0.0

In [114]:
#export


def make_records_df(sources, connected_vertices):
    return pd.DataFrame.from_records(
        [
            {"source": src, "destination": dst, "edge_type": "repo-file"}
            for (src, destinations) in zip(sources, connected_vertices)
            for dst in destinations 
        ]
    )

In [115]:
train_records_df = make_records_df(graphsage_data_train.repos, graph_data_train.X.fillna("").str.split()).drop_duplicates()

In [116]:
#export

def get_vertex_embeddings(wrapper, vertex_subset, model):
    features = (
        model.full_forward(
            wrapper.dataset.x, wrapper.dataset.edge_index
        )
        #.cpu()
        .detach()
        .numpy()
    )
    return features[wrapper.vertex_mapping.loc[vertex_subset]]

In [117]:
other_records_df = make_records_df(graphsage_data_train.repos, graph_data_train.X.dropna().str.split())

In [118]:
dep_graph_df = pd.concat([non_root_dependency_records_df, make_records_df(graphsage_data_train.repos, graph_data_train.X.dropna().str.split())])

In [119]:
%%time
extended_dependency_graph_wrapper = PygGraphWrapper(embeddings.FastTextVectorizer(fasttext_model), dep_graph_df)

CPU times: user 44.9 s, sys: 3.98 s, total: 48.9 s
Wall time: 44.4 s


In [120]:
extended_dependency_graph_wrapper.dataset.x.device

device(type='cpu')

In [121]:
#export
import pathlib



def make_pyggraph_retriever_learner(zs_learner, dependency_graph_wrapper, model):
    
    lambda_transformer = PyGGraphModelTransformer(model, dependency_graph_wrapper)
    return RetrieverLearner(
        zs_learner,
        lambda_transformer,
        dependency_graph_wrapper.featurizer
    )


def save_pyggraph_retriever_learner(pyggraph_retriever_learner, directory):
    p = pathlib.Path(directory)
    p.mkdir(exist_ok=True)

In [122]:
graphsage_learner = make_pyggraph_retriever_learner(
    zero_shot.ESZSLearner(100,10),
    dependency_graph_wrapper,
    graphsage_model,
)

In [123]:
graphsage_results = run_learner_experiment(graphsage_learner, graphsage_data_train, graphsage_data_test)

/home/kuba/Projects/github_search/github_search/pytorch_geometric_data.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.dataset = Data(torch.tensor(features), torch.tensor(edge_index))
/home/kuba/Projects/github_search/github_search/pytorch_geometric_data.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.dataset = Data(torch.tensor(features), torch.tensor(edge_index))


# Results 

In [124]:
s = 'zero shot learning'

In [125]:
graphsage_data_test.y.unique()

array(['dictionary learning', 'natural language inference',
       'facial landmark detection', 'q learning', 'reconstruction',
       'knowledge distillation', 'task oriented dialogue systems',
       'outlier detection', 'meta learning', 'super resolution',
       'image to image translation', 'neural architecture search',
       'style transfer', 'text summarization',
       'interpretable machine learning', 'natural language understanding',
       'policy gradient methods', 'multi agent reinforcement learning',
       'self supervised learning', 'self driving cars',
       'sentence classification', 'reading comprehension',
       'object detection', 'speaker verification', 'multi armed bandits',
       'scene text', 'image inpainting', 'network embedding',
       'demosaicking', 'person re identification',
       'out of distribution detection', 'robust classification',
       'link prediction', 'few shot learning', 'network pruning',
       'multi task learning', 'edge computing'

In [148]:
graphsage_data_test.all_tasks.explode().unique()

array(['dictionary learning', 'answer selection',
       'natural language inference', 'paraphrase identification',
       'facial landmark detection', 'combinatorial optimization',
       'q learning', 'continuous control', 'autonomous driving',
       'reconstruction', 'knowledge distillation', 'topic models',
       'task oriented dialogue systems',
       'abnormal event detection in video', 'outlier detection', 'automl',
       'meta learning', 'deblurring', 'image super resolution',
       'super resolution', 'image manipulation',
       'image to image translation',
       'multimodal unsupervised image to image translation',
       'style transfer', 'unsupervised image to image translation',
       'automated feature engineering', 'hyperparameter optimization',
       'neural architecture search',
       'hyperspectral image super resolution',
       'extractive text summarization', 'text summarization',
       'interpretable machine learning', 'l2 regularization',
       'stoc

In [150]:
results_df = pd.DataFrame.from_records([abstract_results, readme_results, prone_results, graphsage_results])
results_df['method'] = ['abstract', 'readme', 'prone', 'graphsage']

In [152]:
results_df.round(3)

,accuracy_train,accuracy_test,top10_accuracy_train,top10_accuracy_test,method
0,0.533,0.140,0.879,0.736,abstract
1,0.389,0.059,0.824,0.620,readme
2,0.172,0.010,0.479,0.100,prone
3,0.166,0.014,0.441,0.298,graphsage


In [127]:
results_df = pd.DataFrame.from_records([abstract_results, readme_results, graphsage_results])
results_df['method'] = ['abstract', 'readme', 'graphsage']
results_df.round(3)

,accuracy_train,accuracy_test,top10_accuracy_train,top10_accuracy_test,method
0,0.533,0.140,0.879,0.736,abstract
1,0.389,0.059,0.824,0.620,readme
2,0.166,0.014,0.441,0.298,graphsage


In [128]:
from IPython import display

In [129]:
results_df.round(3).to_csv("output/retrieval_results.csv")

In [130]:
#export


def get_query_level_results(
    retriever_learner,
    data_test
):
    
    accuracy_test = retriever_learner.evaluate(data_test, metrics.accuracy_score)
    results = get_retrieval_accuracies(retriever_learner, data_test, k=10)
    results['position'] = results['position'].replace(-1, np.inf)
    
    return accuracy_test, results

In [203]:
#export
def get_idx_or_inf(xs, a):
    idxs = np.where(xs == a)[0].astype(int)
    if len(idxs) == 0:
        return np.inf
    else:
        return idxs[0]
    

def get_areas(area_grouped_tasks, tasks):
    return tasks.apply(
        lambda ts:
        area_grouped_tasks['area'][
            area_grouped_tasks['task'].isin(ts)
        ].unique()
    )


def analyze_query_level_results(query_level_results, area_grouped_tasks):
    retrieval_results_with_area_test = area_grouped_tasks.merge(query_level_results, left_on='task', right_index=True)
    for tasks in retrieval_results_with_area_test['retrieved_labels'].values:
        for task in tasks:
            try:
                partial(get_areas, area_grouped_tasks)([task])
            except:
                print(task)
    retrieved_areas = get_areas(area_grouped_tasks, retrieval_results_with_area_test['retrieved_labels'])#apply(partial(get_areas, area_grouped_tasks))
    retrieval_results_with_area_test['retrieved_areas'] = retrieved_areas
    is_area_retrieved = retrieval_results_with_area_test.apply(lambda row: row['area'] in row['retrieved_areas'][:10], axis=1)
    num_area_retrieved = retrieval_results_with_area_test.apply(lambda row: len(np.where(row['area'] == np.array(row['retrieved_areas'])[:10])[0]), axis=1)
    area_idx = retrieval_results_with_area_test.apply(lambda row: get_idx_or_inf(np.array(row['retrieved_areas']), row['area']), axis=1)
    retrieval_results_with_area_test['area_recalled'] = is_area_retrieved
    retrieval_results_with_area_test['area_position'] = area_idx
    retrieval_results_with_area_test['num_area_recalled'] = num_area_retrieved
    query_level_results = retrieval_results_with_area_test.groupby('area').agg(
        {
            "recalled": "mean",
            "num_recalled": "mean",
            "position": ["median", "mean"],
            "area_recalled": "mean",
            "num_area_recalled": "mean",
            "area_position": ["median", "mean"],
        }
    )
    query_level_results['count'] = retrieval_results_with_area_test['area'].value_counts()#groupby('area').agg('count')
    return query_level_results 
    
def get_analyzed_query_level_results(retriever_learner, data_test, area_grouped_tasks):
    detailed_results_all = get_query_level_results(retriever_learner, data_test)
    retrieval_results_with_area_test = analyze_query_level_results(query_level_results, area_grouped_tasks)
    return retrieval_results_with_area_test 

In [200]:
readme_accuracy, raw_readme_area_results = get_query_level_results(readme_learner, readme_data_test)

KeyboardInterrupt: 

In [ ]:
graphsage_accuracy, raw_graphsage_area_results = get_query_level_results(graphsage_learner, graphsage_data_test)

In [163]:
readme_accuracy

0.059114735002912054

In [164]:
readme_data_test.y

0               dictionary learning
6        natural language inference
7        natural language inference
8        natural language inference
9        natural language inference
                    ...            
17378    neural network compression
17379    neural network compression
17384                style transfer
17385                style transfer
17386                style transfer
Name: most_common_task, Length: 3434, dtype: object

In [165]:
graphsage_accuracy

0.013508949679162444

In [170]:
raw_readme_area_results

,retrieved_labels,num_recalled,recalled,position
dictionary learning,"[incremental learning, knowledge distillation,...",1,True,2.0
natural language inference,"[abstractive text summarization, text summariz...",3,True,2.0
facial landmark detection,"[face alignment, face recognition, face recons...",0,False,inf
q learning,"[atari games, q learning, multi goal reinforce...",3,True,1.0
reconstruction,"[image compression, ms ssim, neural architectu...",1,True,16.0
...,...,...,...,...
real time strategy games,"[link prediction, atari games, q learning, wea...",0,False,inf
time series classification,"[audio classification, weather forecasting, li...",0,False,inf
few shot image classification,"[interpretable machine learning, image to imag...",0,False,inf
image super resolution,"[image super resolution, super resolution, ima...",10,True,0.0


In [180]:
ras

NameError: name 'ras' is not defined

In [181]:
s = "  "

2

In [179]:
all_tasks_tasks = all_tasks.explode().unique()

all_tasks_tasks[~pd.Series(all_tasks_tasks).isin(area_grouped_tasks['task'])]

array(['lidar semantic segmentation', 'self supervised video retrieval',
       'the semantic segmentation of remote sensing imagery',
       'persian sentiment analysis', 'few shot semantic segmentation',
       'node clustering', 'image morphing',
       'no reference image quality assessment', 'landmark recognition',
       'interestingness detection', 'extreme summarization',
       'few shot text classification', 'cube engraving classification',
       'video quality assessment', 'multi label zero shot learning',
       'portrait segmentation', 'unsupervised sentence compression',
       'indoor scene understanding',
       'logical reasoning question answering',
       'covid 19 image segmentation', 'nonhomogeneous image dehazing',
       'text segmentation', 'jpeg compression artifact reduction',
       'single view  reconstruction', 'volumetric reconstruction',
       'cryogenic electron microscopy (cryo em)',
       'spinal cord gray matter   segmentation', 'music genre transf

In [172]:
all_raw_tasks = raw_graphsage_area_results.retrieved_labels.explode().drop_duplicates().values

In [173]:
all_raw_tasks[~pd.Series(all_raw_tasks).isin(area_grouped_tasks['task'])]

array(['multi label zero shot learning', '', 'object skeleton detection',
       'indoor scene understanding', 'bird view synthesis',
       'music genre transfer', 'single view  reconstruction',
       'disaster response', 'music style transfer'], dtype=object)

In [186]:
area_grouped_tasks['task'].isin() 

TypeError: isin() missing 1 required positional argument: 'values'

In [195]:
area_grouped_tasks

,area,task
0,adversarial,adversarial text
1,adversarial,adversarial attack
2,adversarial,adversarial defense
3,adversarial,inference attack
4,adversarial,data poisoning
...,...,...
1875,miscellaneous,lexical entailment
1877,miscellaneous,classification consistency
1878,miscellaneous,foveation
1879,miscellaneous,unsupervised video object segmentation


In [196]:
retrieval_results_with_area_test['retrieved_labels']

2       [adversarial defense, adversarial defense, sel...
22      [interpretable machine learning, image to imag...
25      [audio generation, speech synthesis, style tra...
42      [dependency parsing, semantic parsing, atari g...
88      [q learning, atari games, board games, multi t...
                              ...                        
1768    [image to image translation, image to image tr...
1776    [meta learning, multi agent reinforcement lear...
1780    [face identification, face recognition, face v...
1784    [depth estimation, super resolution, image sup...
1803    [fine grained opinion analysis, multi task lea...
Name: retrieved_labels, Length: 148, dtype: object

In [198]:

[area_grouped_tasks['area'][
        area_grouped_tasks['task'].isin(ts)]
    for ts in retrieval_results_with_area_test['retrieved_labels']
    ]



[1           adversarial
 2           adversarial
 169     computer-vision
 224     computer-vision
 564     computer-vision
 923         methodology
 1470      playing-games
 1541             speech
 1550             speech
 1586             speech
 1741      miscellaneous
 1783      miscellaneous
 Name: area, dtype: object,
 2           adversarial
 22                audio
 225     computer-vision
 716             medical
 800             medical
 958         methodology
 1541             speech
 1550             speech
 1586             speech
 1678      miscellaneous
 1741      miscellaneous
 1746      miscellaneous
 1768      miscellaneous
 1858      miscellaneous
 Name: area, dtype: object,
 25                audio
 112     computer-vision
 225     computer-vision
 297     computer-vision
 431     computer-vision
 477     computer-vision
 481     computer-vision
 571     computer-vision
 756             medical
 1554             speech
 1559             speech
 1585             s

In [202]:
_query_level_results = raw_readme_area_results
retrieval_results_with_area_test = area_grouped_tasks.merge(_query_level_results, left_on='task', right_index=True)

retrieved_areas = retrieval_results_with_area_test['retrieved_labels'].apply(partial(get_areas, area_grouped_tasks))
retrieved_areas
#retrieval_results_with_area_test['retrieved_areas'] = retrieved_areas
#is_area_retrieved = retrieval_results_with_area_test.apply(lambda row: row['area'] in row['retrieved_areas'][:10], axis=1)
#num_area_retrieved = retrieval_results_with_area_test.apply(lambda row: len(np.where(row['area'] == np.array(row['retrieved_areas'])[:10])[0]), axis=1)
#area_idx = retrieval_results_with_area_test.apply(lambda row: get_idx_or_inf(np.array(row['retrieved_areas']), row['area']), axis=1)
#retrieval_results_with_area_test['area_recalled'] = is_area_retrieved
#retrieval_results_with_area_test['area_position'] = area_idx
#retrieval_results_with_area_test['num_area_recalled'] = num_area_retrieved

AttributeError: 'numpy.ndarray' object has no attribute 'apply'

In [205]:
readme_area_results = analyze_query_level_results(raw_readme_area_results, area_grouped_tasks)

adversarial defense
adversarial defense
self supervised learning
adversarial defense
adversarial defense
adversarial defense
robust classification
speaker identification
speaker recognition
speaker verification
adversarial attack
person re identification
adversarial attack
image retrieval
atari games
efficient exploration
binarization
robust classification
interpretable machine learning
image to image translation
multimodal unsupervised image to image translation
style transfer
unsupervised image to image translation
audio classification
adversarial defense
speaker identification
speaker recognition
speaker verification
noise estimation
robust classification
lesion segmentation
two sample testing
two sample testing
style transfer
audio generation
speech synthesis
style transfer
cross view image to image translation
fundus to angiography generation
image to image translation
nuclear segmentation
image super resolution
super resolution
text to speech synthesis
image captioning
image to i

In [215]:
readme_area_results.round(2).sort_values('count', ascending=False)

recalled num_recalled position         \
                                mean         mean   median   mean   
area                                                                
miscellaneous                   0.54         1.56      7.0    inf   
computer-vision                 0.58         2.03     15.0    inf   
natural-language-processing     0.58         1.69     10.5    inf   
methodology                     0.62         1.88      5.0    inf   
graphs                          0.80         2.00      3.0    inf   
speech                          1.00         2.50      0.5   0.75   
medical                         0.67         2.33      0.0    inf   
playing-games                   0.67         1.67      4.0    inf   
audio                           1.00         1.00      2.5   2.50   
robots                          1.00         1.50      7.5   7.50   
time-series                     0.00         0.00      inf    inf   
adversarial                     1.00         5.00      0.0   0.00   
computer-code                   1.00         2.00     14.0  14.00   
knowledge-base                  1.00         5.00      1.0   1.00   

                            area_recalled num_area_recalled area_position  \
                                     mean              mean        median   
area                                                                        
miscellaneous                        0.95              0.95           3.0   
computer-vision                      0.97              0.97           0.0   
natural-language-processing          0.88              0.88           3.0   
methodology                          0.96              0.96           1.0   
graphs                               1.00              1.00           1.0   
speech                               1.00              1.00           2.0   
medical                              1.00              1.00           1.0   
playing-games                        1.00              1.00           2.0   
audio                                1.00              1.00           0.5   
robots                               1.00              1.00           2.0   
time-series                          0.00              0.00           inf   
adversarial                          1.00              1.00           0.0   
computer-code                        1.00              1.00           0.0   
knowledge-base                       1.00              1.00           2.0   

                                  count  
                             mean        
area                                     
miscellaneous                 inf    41  
computer-vision               inf    33  
natural-language-processing   inf    26  
methodology                   inf    24  
graphs                       0.80     5  
speech                       2.50     4  
medical                      1.33     3  
playing-games                3.00     3  
audio                        0.50     2  
robots                       2.00     2  
time-series                   inf     2  
adversarial                  0.00     1  
computer-code                0.00     1  
knowledge-base               2.00     1

In [208]:
readme_area_results.mean()

recalled           mean       0.746257
num_recalled       mean       2.154185
position           median          inf
                   mean            inf
area_recalled      mean       0.911705
num_area_recalled  mean       0.911705
area_position      median          inf
                   mean            inf
count                        10.571429
dtype: float64

In [209]:
graphsage_area_results = analyze_query_level_results(raw_graphsage_area_results, area_grouped_tasks)

word sense disambiguation
feature selection
image inpainting
audio classification
policy gradient methods
image retrieval
metric learning
person re identification
density estimation
latent variable models
normalising flows
image restoration
image super resolution
super resolution
scene text
scene text recognition
semi supervised image classification
image super resolution
super resolution
multi agent reinforcement learning
q learning
self supervised learning
autonomous vehicles
federated learning
meta learning
multi task learning
style transfer
openai gym
q learning
multi agent reinforcement learning
image inpainting
image super resolution
super resolution
optical character recognition
scene text
scene text recognition
multi agent reinforcement learning
autonomous driving
multi task learning
curved text detection
scene text
scene text detection
brain tumor segmentation
lesion segmentation
tumor segmentation
omniglot
answer selection
natural language inference
paraphrase identification


In [214]:
graphsage_area_results.round(2).sort_values('count', ascending=False)

recalled num_recalled position      area_recalled  \
                                mean         mean   median mean          mean   
area                                                                            
miscellaneous                   0.17         0.23      inf  inf          0.91   
computer-vision                 0.34         0.59      inf  inf          1.00   
natural-language-processing     0.25         0.46      inf  inf          0.96   
methodology                     0.38         0.81      inf  inf          1.00   
graphs                          0.25         0.25      inf  inf          0.75   
speech                          0.50         0.50      inf  inf          1.00   
medical                         0.33         1.00      inf  inf          0.67   
playing-games                   0.00         0.00      inf  inf          1.00   
time-series                     0.00         0.00      inf  inf          0.00   
audio                           1.00         1.00      3.0  3.0          1.00   
computer-code                   0.00         0.00      inf  inf          0.00   
knowledge-base                  0.00         0.00      inf  inf          0.00   
robots                          0.00         0.00      inf  inf          0.00   

                            num_area_recalled area_position       count  
                                         mean        median  mean        
area                                                                     
miscellaneous                            0.91           3.0   inf    35  
computer-vision                          1.00           0.0  0.12    32  
natural-language-processing              0.96           4.0   inf    24  
methodology                              1.00           2.0  1.95    21  
graphs                                   0.75           1.5   inf     4  
speech                                   1.00           5.0  4.75     4  
medical                                  0.67           2.0   inf     3  
playing-games                            1.00           2.0  2.00     2  
time-series                              0.00           inf   inf     2  
audio                                    1.00           0.0  0.00     1  
computer-code                            0.00           inf   inf     1  
knowledge-base                           0.00           inf   inf     1  
robots                                   0.00           inf   inf     1

In [ ]:
graphsage_area_results.mean()

In [ ]:
analyzed_readme_results = analyze_query_level_results(rea, area_grouped_tasks)

In [ ]:
analyzed_readme_results

In [ ]:
analyzed_graphsage_results = analyze_query_level_results(detailed_results_graphsage_all, area_grouped_tasks)

In [ ]:
analyzed_readme_results.sort_values(('recalled','mean'))

In [ ]:
analyzed_graphsage_results.sort_values(('recalled','mean'))

In [ ]:
area_grouped_tasks['task'] == [detailed_results_readme['top10_accuracy_test'].index]

In [ ]:
detailed_results_readme['top10_accuracy_test'].groupby(area_grouped_tasks['task'].loc[detailed_results_readme['top10_accuracy_test'].index])

# Exporting models 

In [ ]:
pickle.dump(readme_data_test, open("output/readme_data_test.pkl", "wb"))

In [ ]:
pickle.dump(graphsage_data_test, open("output/graphsage_data_test.pkl", "wb"))

In [ ]:
pickle.dump(graphsage_learner, open("output/graphsage_learner.pkl", "wb"))

In [ ]:
pickle.dump(readme_learner, open("output/readme_learner.pkl", "wb"))

In [ ]:
readme_retriever = Retriever.from_retriever_learner(readme_learner)
readme_retriever.set_embeddings(readme_data_train.repos, readme_data_train.X)

In [ ]:
graphsage_retriever = Retriever.from_retriever_learner(graphsage_learner)
graphsage_retriever.set_embeddings(graphsage_data_train.repos, graphsage_data_train.X)

In [ ]:
%%time
metric_learning_results = readme_retriever.retrieve_query_results("metric learning", k=5).values

In [ ]:
%%time
metric_learning_results_graphsage = graphsage_retriever.retrieve_query_results("metric learning", k=5).values

In [ ]:
distance_learning_results = readme_retriever.retrieve_query_results("distance learning", k=5).values

In [ ]:
distance_learning_results_graphsage = graphsage_retriever.retrieve_query_results("distance learning", k=5).values

In [ ]:
distance_learning_results

In [ ]:
distance_learning_results_graphsage

In [ ]:
readme_retriever.retrieve_query_results("video text detection", k=10).values

In [ ]:
graphsage_retriever.retrieve_query_results("video text detection", k=10).values

In [ ]:
readme_retriever.retrieve_query_results("context recommender systems", k=10).values

In [ ]:
graphsage_retriever.retrieve_query_results("context recommender systems", k=10).values

In [ ]:
readme_retriever.retrieve_query_results("bayesian optimization", k=10).values

In [ ]:
graphsage_retriever.retrieve_query_results("bayesian optimization", k=10).values

In [ ]:
readme_retriever.retrieve_query_results("evolutionary methods", k=10).values

In [ ]:
graphsage_retriever.retrieve_query_results("evolutionary methods", k=10).values

In [ ]:
readme_retriever.retrieve_query_results("painting", k=10).values

In [ ]:
graphsage_retriever.retrieve_query_results("painting", k=10).values

In [ ]:
%%time
retrieve_query_results(graphsage_learner, graphsage_data_train.repos, graphsage_data_train.X, "distance learning").values

In [ ]:
retrieve_query_results(graphsage_learner, graphsage_data_train.X, "distance learning")

# Concatenation of repo, import embeddings

In [ ]:
paired_data_train, paired_data_test = RepoTaskData.create_split(tasks_train, all_tasks, paperswithcode_with_features_df, paperswithcode_with_imports_df['imports'])
paired_data_train.X = graph_data_train.X + " " + import_data_train.X
paired_data_test.X = graph_data_test.X + " " + import_data_test.X

In [ ]:
paired_data_train.X

In [ ]:
paired_learner = RetrieverLearner.create(
    zero_shot.ESZSLearner(100, 10),
    PairedKeyedVectors(python_word_embeddings.wv, graphsage_embeddings),
    fasttext_model,
    embeddings.AverageWordEmbeddingsVectorizer,
    embeddings.FastTextVectorizer
)

paired_learner.fit_learner(graph_data_train)

In [ ]:
paired_learner.evaluate(graph_data_train, metric=metrics.accuracy_score)

In [ ]:
get_retrieval_accuracy(paired_learner, paired_data_train, k=10)

In [ ]:
get_retrieval_accuracy(paired_learner, paired_data_test, k=10)

In [ ]:
results = []
for (learner, learner_name, test) in zip(
    [import2vec_learner, prone_learner, paired_learner],
    ['import2vec', 'prone', 'both'],
    [X_test, repo_graph_terms_test, X_paired_test]
):
    accs = []
    for k in [1, 3, 5, 10, 20]:
        rec = get_retrieval_accuracy(learner, test, y_test, test_task_idxs, k=k)
        accs.append(rec)
    results.append(pd.Series(name=learner_name, data=accs))

In [ ]:
results_df = pd.DataFrame(results)
results_df.columns = ["Accuracy@{}".format(i) for i in [1, 3, 5, 10, 20]]

In [ ]:
results_df.round(3).to_markdown(open("metrics/zsl_results.md", "w"))

In [ ]:
!cat metrics/zsl_results.md

In [ ]:
import toolz

In [ ]:
task_distances = metrics.pairwise.cosine_distances(task_embeddings, task_embeddings)

In [ ]:
poincare_embeddings = gensim.models.KeyedVectors.load('data/poincare5.vec')

In [ ]:
import gensim.models.wrappers.fasttext
from gensim.test.utils import datapath

In [ ]:
from github_search import typical_file_parts
from mlutil import prototype_selection

In [ ]:
selected_lines_df = typical_file_parts.get_selected_lines_and_repos(python_files_df['repo_name'], python_files_df['content'])

# Selecting prototypical lines

In [ ]:
fasttext_selector = prototype_selection.PrototypeSelector(fasttext_avg_embedder)

In [ ]:
try:
    fasttext_prototypes = json.load(open('data/fasttext_prototypes.json', 'r'))
except:
    fasttext_selector.fit_prototypes(selected_lines_df['line'], selected_lines_df['repo'])
    fasttext_prototypes = fasttext_selector.prototypes
    json.dump(fasttext_prototypes, open('data/fasttext_prototypes.json', 'w'))

In [ ]:
codebert_vectorizer = embeddings.TransformerVectorizer('microsoft/codebert-base', batch_size=64)

In [ ]:
codebert_selector = prototype_selection.PrototypeSelector(codebert_vectorizer)

In [ ]:
try:
    codebert_prototypes = json.load(open('data/codebert_prototypes.json', 'r'))
except:
    codebert_selector.fit_prototypes(selected_lines_df['line'], selected_lines_df['repo'])
    codebert_prototypes = codebert_selector.prototypes
    json.dump(codebert_prototypes, open('data/codebert_prototypes.json', 'w'))

In [ ]:
def vectorize_prototypes(vectorizer, prototypes):
    prototype_aggregated_embeddings = {}
    for key in prototypes.keys():
        prototype_aggregated_embeddings[key] = np.mean(vectorizer.transform(prototypes[key]), axis=0)
    return list(prototype_aggregated_embeddings.keys()), np.row_stack(prototype_aggregated_embeddings.values())

In [ ]:
codebert_prototypes = {
    repo: v
    for (repo, v) in codebert_prototypes.items()
    if repo in paperswithcode_with_imports_df['repo_name'].values
}

In [ ]:
codebert_prototypes.keys()

In [ ]:
repos_train

In [ ]:
fasttext_prototypes = {
    repo: v
    for (repo, v) in fasttext_prototypes.items()
    if repo in paperswithcode_with_imports_df['repo_name'].values
}

In [ ]:
def get_prototypes(repo_name):
    return pd.DataFrame({"codebert": codebert_prototypes[repo_name], "fasttext": fasttext_prototypes[repo_name]})

In [ ]:
fasttext_prototypes.keys()

In [ ]:
get_prototypes("transformer")

In [ ]:
get_prototypes("mmdetection")

In [ ]:
get_prototypes("Recommenders-movielens")

In [ ]:
get_prototypes("mmdetection")

In [ ]:
fasttext_prototypes['mmdetection']

In [ ]:
codebert_repos, codebert_prototype_embeddings = vectorize_prototypes(codebert_vectorizer, codebert_prototypes)

In [ ]:
fasttext_repos, fasttext_prototype_embeddings = vectorize_prototypes(fasttext_avg_embedder, fasttext_prototypes)

In [ ]:
len(fasttext_prototype_embeddings)

In [ ]:
paperswithcode_tasks_series = paperswithcode_with_imports_df['most_common_task']
paperswithcode_tasks_series.index = paperswithcode_with_imports_df['repo_name']
#paperswithcode_tasks_series = paperswithcode_tasks_series[paperswithcode_tasks_series.index.isin(fasttext_repos)]

In [ ]:
fasttext_tasks = paperswithcode_tasks_series.loc[fasttext_repos]
fasttext_tasks_embeddings = task_embedder.transform(fasttext_tasks)
codebert_tasks = paperswithcode_tasks_series.loc[codebert_repos]
codebert_tasks_embeddings = task_embedder.transform(codebert_tasks)

In [ ]:
codebert_prototype_embeddings.shape

In [ ]:
eszs_learner = zero_shot.ESZSLearner()

In [ ]:
codebert_prototype_embeddings.shape

In [ ]:
len(codebert_tasks)

In [ ]:
eszs_learner.fit(codebert_prototype_embeddings, codebert_tasks, task_embeddings[:-1])
eszs_learner.score(codebert_prototype_embeddings, codebert_tasks, task_embeddings[:-1])

In [ ]:
eszs_learner.fit(fasttext_prototype_embeddings, fasttext_tasks, task_embeddings[:-1])
eszs_learner.score(fasttext_prototype_embeddings, fasttext_tasks, task_embeddings[:-1])

In [ ]:
list(set(selected_lines_df['repo']))[3007]

In [ ]:
problematic_lines_df = selected_lines_df[selected_lines_df['repo'] == 'auto_ml']

In [ ]:
del codebert_vectorizer

In [ ]:
problematic_lines_df['lines']

In [ ]:
codebert_selector.prototypes

In [ ]:
y_embeddings = fasttext_avg_embedder.transform(tasks)

In [ ]:
repo_names = 
repo_embeddings = 

In [ ]:
y_embeddings.shape